진호님이 일단 실행되게끔 만들어 본것

In [2]:
# !pip install konlpy

In [1]:
# 3_21 모델학습 후 예측한 결과가 심통치않아 새로 전처리 해보기로 하였다.
# 벡터를 이용한 모델을 생성해서 문자열 값을 리턴 받아보자.
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
import tensorflow as tf
import numpy as np

#1. Naver Sentiment Movie Corpus v1.0 다운로드
train_file = tf.keras.utils.get_file('train.txt', 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')
test_file = tf.keras.utils.get_file( 'test.txt',  'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt')

#파일을 이진모드로 읽어온다. 디코드는 utf8로한다.
train_text = open(train_file,'rb').read().decode(encoding='utf-8')
test_text = open(test_file,'rb').read().decode(encoding='utf-8')



In [2]:
import pandas as pd
#x,y값을 추출후 데이터프레임에 넣는다.
# X 값 추출 피쳐feature, y값 추출target 

df_train = pd.DataFrame({
    'feature':[row.split('\t')[1] for row in train_text.split('\n')[1:] if row.count('\t')>0],
    'target': [ int(row.split('\t')[2]) for row in train_text.split('\n')[1:] if row.count('\t')>0]
})


In [3]:
# df_train.iloc[:,0]
from konlpy.tag import Okt

#문장을 토큰화한것만 따로 저장
# tokenized_data =[]

# for i in df_train.iloc[:,0]:
#     tokenized_data.append(Okt().morphs( i ))

In [4]:
import pickle

#토근화된 데이터를 저장하였음
# with open('tokenized_data.pkl','wb') as f:
#     pickle.dump(tokenized_data,f)

#토큰화된 데이터 불러오기
# with open('tokenized_data.pkl','rb') as f:
#     tokenized_data_loaded = pickle.load(f)

# tokenized_data_loaded

In [5]:
#라벨별로 분리해본것
bad = df_train[df_train.iloc[:,1]==0]
good = df_train[df_train.iloc[:,1]==1]


df_train = df_train[::5]

from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Okt

vectorizer=TfidfVectorizer(tokenizer=Okt().morphs)
X = vectorizer.fit_transform( df_train.iloc[:,0])
y = df_train.iloc[:,1]


c:\Users\uk246\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [8]:
df_train[df_train.iloc[:,1]==0]  #18678 
df_train[df_train.iloc[:,1]==1]  #18822
df_train.shape

(37500, 2)

In [9]:
# 토큰화하고 빈도수(tfidf)에 기반한 벡터표현이 생성된 X를 따로저장
from scipy.sparse import save_npz ,load_npz

#vectorizer.fit_transform한것을 희소행렬로로 저장
# save_npz('37500개_vectorized_data.npz', X)

#저장된 희소행렬을 불러오기
# X_loaded = load_npz('37500개_vectorized_data.npz')

#이렇게 불러온 X_loaded는 바로 모델학습에 사용가능


In [10]:
import numpy as np
X_train_dense = np.expand_dims( X.toarray(), axis=-1 )
# X_train_dense.shape
# X_loaded.shape[1]
# X_loaded.toarray().shape[1]

In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization , Input, Dropout
# from tensorflow.keras import Input
# 모델 구성deep nueral network
# input_layer = Input(shape=(X_loaded.shape[1],), sparse=True)

#flatten 사용하는것 알아보기

model = Sequential([
    # input_layer,
    Input(shape=(X_train_dense.shape[1],)),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.6),
    Dense(32, activation='relu'),
    BatchNormalization(),
    Dropout(0.6),
    Dense(1, activation='sigmoid')
])

# 모델 컴파일 및 학습
model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

# model.fit(X.toarray(), y, epochs = 10, batch_size = 500, validation_split=0.2)  #0.8417
model.fit(X.toarray(), y, epochs = 10, batch_size = 20, validation_split=0.3)  #0.8417

Epoch 1/10
1313/1313 [==============================] - 46s 31ms/step - loss: 0.5839 - accuracy: 0.7071 - val_loss: 0.4358 - val_accuracy: 0.8067
Epoch 2/10
1313/1313 [==============================] - 31s 23ms/step - loss: 0.4466 - accuracy: 0.7986 - val_loss: 0.3997 - val_accuracy: 0.8213
Epoch 3/10
1313/1313 [==============================] - 31s 24ms/step - loss: 0.3906 - accuracy: 0.8346 - val_loss: 0.4041 - val_accuracy: 0.8149
Epoch 4/10
1313/1313 [==============================] - 29s 22ms/step - loss: 0.3596 - accuracy: 0.8528 - val_loss: 0.3872 - val_accuracy: 0.8245
Epoch 5/10
1313/1313 [==============================] - 31s 23ms/step - loss: 0.3287 - accuracy: 0.8686 - val_loss: 0.4150 - val_accuracy: 0.8080
Epoch 6/10
1313/1313 [==============================] - 29s 22ms/step - loss: 0.3157 - accuracy: 0.8730 - val_loss: 0.4016 - val_accuracy: 0.8238
Epoch 7/10
1313/1313 [==============================] - 34s 26ms/step - loss: 0.3192 - accuracy: 0.8717 - val_loss: 0.4511 -

In [107]:
test = ['정말 별로 입니다. 이게뭐니']

# 학습된 TF-IDF 벡터라이저로 테스트 데이터 변환
X_test_sample = vectorizer.transform(test)
X_test_sample_dense = X_test_sample.toarray()

# 모델을 사용하여 예측 수행
predictions = model.predict(X_test_sample_dense)
print(predictions)

# 예측 결과 출력 (긍정: 1, 부정: 0)
print("예측 결과:", np.round( predictions ).astype(int))

1/1 [==============================] - 0s 49ms/step
[[0.01700829]]
예측 결과: [[0]]


닉네임기준으로 
카톡대화내용을 기준으로 테스트 문장을 여러개 불러와서 예측 후 그 값의 비율을 계산 출력해준다.

In [25]:
# model.save('기본 DNN 적용모델.h5')